### Integrantes
- José Martinez
- Rodrigo Montecinos.

# En lo que sigue se importarán las librerias necesarias.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Creamos los valores aleatorios para poder rankear los productos.

In [ ]:
### Preferencia para probar el producto a_{i}
def appeal(n):
  a = np.random.uniform(0,1,n)
  a /= np.sum(a)
  return a

### Generamos q_{i}, la probabilidad condicional de comprar el producto i dado que
### el cliente lo probó.

def quality(n):
  q = np.random.uniform(0,1,n)
  q = np.sort(q)[::-1]
  q /= np.sum(q)
  return q

### Generamos el vector de visibilidad
def visibility(n):
  v = np.random.uniform(0,1,n)
  v = np.sort(v)[::-1]
  v /= np.sum(v)
  return v


## Supondremos que estamos en el tiempo t, en el que se realiza una acción, es decir, el cliente en el tiempo t puede decidir si comprar unos de los productos o no. Para esto escribiremos el vector d que posee la cantidad de veces que a comprado cada producto i en el tiempo t.

In [ ]:
### En tiempo inicial 0 definiremos el vector correspondiente a 0 ventas. (dispuesto a cambios)

def purchased_products(n): # Son la cantidad de compras y la cantidad de productos.
  d = np.zeros(n ,dtype = int)
  return d

## Se crea un vector que le da un ranking aleatorio al producto i, por ejemplo si la posición j tiene un valor k, entonces el producto j+1 está rankeado en la posición k.

In [ ]:
def ranking_aleatorio(n):
    # Creamos una secuencia de números del 1 al n.
    numeros = list(range(n))
    # Mezcla la secuencia de forma aleatoria.
    random.shuffle(numeros)
    sigma = np.array(numeros)

    ## Creamos el vector final que posee dos coordenadas.
    #sigma = np.array([[i+1,numeros[i]] for i in range(n)])
    return sigma

rank = ranking_aleatorio(10)
print(rank)


[5 0 9 8 7 3 2 1 4 6]


## Se crea la función de probabilidad de que el cliente en el tiempo t va a probar el producto i dado un ranking sigma.

In [ ]:
n = 4
P=np.array([[0.6, 0.05, 0.05, 0.25],[0.15, 0.8, 0, 0.1],[0.25, 0.15, 0.95, 0.4], [0, 0, 0, 0.25]])
mu=np.array([0.15, 0.35, 0.2, 0.3])
sigma = ranking_aleatorio(n)
dd = 10*mu
vis = visibility(n)
a = appeal(n)
def probabilidad_probar_i(i,sigma,d):
  numerador = vis[sigma[i]]*(a[i] + dd[i])
  den = sum([vis[sigma[j]]*(a[j] + dd[j]) for j in range(n)])
  return numerador/den

#p_10 = probabilidad_probar_i(9,sigma,dd)
#print(p_10)
#p_20 = probabilidad_probar_i(9,sigma,dd)
#print(p_20)
### Vamos a verificar que la suma de las probabilidades den 1.
#indices = 0
#suma = 0
#while indices < n:
#  suma += probabilidad_probar_i(indices,sigma,dd)
#  indices += 1

#print(suma)


## Procedemos a definir la función de energía para poder aplicar Simulated annealing.

In [ ]:
def energy_function(P_i,q):
  n = len(q)
  E = sum([P_i[i]*q[i] for i in range(n)])
  return E

q = quality(n)
P_i = np.array([probabilidad_probar_i(i,sigma,dd) for i in range(n)])
print(energy_function(P_i,q))

0.29702398739898783


El grafo que se construye para utilizar simulated annealing es el siguiente:


*   El conjunto de vertices corresponde a las permutaciones de largo n, $S_n$.
*   Dos elementos de $S_n$ están unidos por una arista si difieren en solo dos elementos (ambos elementos están intercambiados).



## La siguiente función entrega una permutación adyacente a $rank$ elegida de manera uniforme.

In [ ]:
def new_ranking(rank):
    n=len(rank);
    i=np.random.randint(n);
    j=np.random.randint(n);
    if i==j:
        j=np.random.randint(n);
    new_rank=rank.copy();
    new_rank[i]=rank[j];
    new_rank[j]=rank[i];
    return new_rank


## Aplicación de simulated annealing

In [ ]:
q=quality(n)
def SA(beta,nf):
  X=ranking_aleatorio(n)
  U=np.random.uniform(size=nf)
  for j in range(1,nf):
    Y=new_ranking(X)
    energy_X=energy_function(X,q)
    energy_Y=energy_function(Y,q)
    if U[j]<=min(np.exp(beta*(energy_X-energy_Y)),1):
      X=Y
    return X



In [ ]:
ejemplo=SA(0.5,1000000)
e1=ranking_aleatorio(n)
print(ejemplo)
print(e1)
print(energy_function(ejemplo,q))
print(energy_function(e1,q))

[0 3 1 2]
[0 3 1 2]
1.4677363180292364
1.4677363180292364
